In [269]:
"""
Adam Nurlign 7/2/2025

Hello there! In this notebook I will be implementing a comprehensive Neural Network Deep Learning Model Framework
in which you can build and customise your own neural networks which can be trained and evaluated on datasets of your choosing.

There are many modules in Python such as PyTorch and Scikit-learn that give you access to machine learning frameworks and
allow you to build your own models made up of layers. I thought i would be a good exercise to be able to implement some of these
features from scratch. I hope you enjoy!

Here are some current constraints to my Machine Learning modules:

-Must have a linear, activation, linear, activation .... linear network structure
-The activations must be sigmoid or relu
-Can only perform stochastic gradient descent

"""

'\nAdam Nurlign 7/2/2025\n\nHello there! In this notebook I will be implementing a comprehensive Neural Network Deep Learning Model Framework\nin which you can build and customise your own neural networks which can be trained and evaluated on datasets of your choosing.\n\nThere are many modules in Python such as PyTorch and Scikit-learn that give you access to machine learning frameworks and\nallow you to build your own models made up of layers. I thought i would be a good exercise to be able to implement some of these\nfeatures from scratch. I hope you enjoy!\n\nHere are some current constraints to my Machine Learning modules:\n\n-Must have a linear, activation, linear, activation .... linear network structure\n-The activations must be sigmoid or relu\n-Can only perform stochastic gradient descent\n\n'

In [270]:
import numpy as np
import pandas as pd

In [271]:
#Layer Superclass
class Layer():
  def __init__(self):
    pass
  def apply(self,x):
    pass

In [272]:
class LinearLayer(Layer):
  def __init__(self,in_dim,out_dim,activation=None,loss=None):
    super().__init__()
    #We should have a matrix with the dimension out_dim by in_dim which draws each element from the normal distribution with mean 0 and variance 4/(in_dim+out_dim)
    variance = 4/(in_dim+out_dim)
    std_dev=np.sqrt(variance)
    self.weights = np.random.normal(loc=0.0, scale=std_dev, size=(out_dim, in_dim))
    self.bias=np.zeros((out_dim,1))
    #storing the derivative of objective with respect to weight matrix
    self.weightGrad=None
    #storing the deravitive of objective with respect to bias vector
    self.biasGrad=None
    #storing the deravitive of objective with respect to this layers output (linear output z)
    self.outputGrad=None

    #storing the forward pass output value
    self.outputValue=None

    self.activation=None
    self.loss=loss

    if (activation=="sigmoid"):
      self.activation=Sigmoid()
    elif (activation=="relu"):
      self.activation=reLU()
    elif (activation=="softmax"):
      self.activation=Softmax()
    else:
      pass


  def apply(self,x):
    return self.weights@x+self.bias

In [273]:
class reLU(Layer):
  def __init__(self):
    super().__init__()
    self.outputValue=None
  def apply(self,x):
    return np.maximum(x,0)

In [274]:
class Sigmoid(Layer):
  def __init__(self):
    super().__init__()
    self.outputValue=None
  def apply(self,x):
    return 1/(1+np.exp(-x))

In [275]:
class Softmax(Layer):
  def __init__(self):
    super().__init__()
    self.outputValue=None
  def apply(self, x):
    x=x.flatten()
    shift_x=x-np.max(x)
    exponent=np.exp(shift_x)
    denominator=np.sum(exponent)
    answer=exponent/denominator
    return answer.reshape(-1,1)



In [276]:
def derivOfActWrtLinearInput(activationType,outputVal):
  if (activationType=="sigmoid"):
    #The deravitive of objective with respect to linear output depends on derivative of activation with respect to linear output which depends on the activation function
    #For the sigmoid function the deravitive with respect to input = input * (1-input)
    fi=np.copy(outputVal)
    oneMinusfi=np.ones(fi.shape)-fi
    derivOfAct=fi*oneMinusfi #this is the deravitive of activation with respect to linera input
    derivOfActMatrix=np.diag(derivOfAct) #has DixDi dimensions where Di is the dimension of input into linear layer
    return derivOfActMatrix
  elif (activationType=="relu"):
    #derivative of relu activation function with respect to input (linear output) is a diagonal matrix of 1's in the diagonal
    #entries where the linear output is >0 and 0 elsewhere
    fi=np.copy(outputVal)
    OnesAndZeroes=(fi>0).astype(int)
    OnesAndZeroes=OnesAndZeroes.flatten()
    OnesAndZeroesMatrix=np.diag(OnesAndZeroes)
    return OnesAndZeroesMatrix
  elif (activationType=="softmax"):
    #Note here unlike the sigmoid and relu activation functions output val will be the activation layers output not linear
    h=outputVal.reshape(-1,1)
    #The derivative of softmax activation with respect to linear input is very simple mathematically, and this is a elegant,
    #complicated expression which does the following operation: on the diagonals i of the matrix returned do hi*1-hi.
    #On the off diagonal representing the deravitive of activation component i with respect to linear input component j do
    #-hi*hj. The matrix we return is symmetric meaning its transpose equals itself so we don't have to worry about whether
    #it is in denominator or numerator format.
    return np.diagflat(h)-np.dot(h,h.T)
  else:
    pass

In [277]:
class NeuralNetwork():
  def __init__(self,layers):
    self.layers=layers

  def predict(self,x,storeValues=False):
    val=np.copy(x)
    if (val.ndim==1):
      val=val.reshape((val.shape[0],1))

    for layer in self.layers:
      val=layer.apply(val)
      if (storeValues==True):
        layer.outputValue=val
      if (layer.activation!=None):
        activationLayer=layer.activation
        val=activationLayer.apply(val)
        if (storeValues==True):
          activationLayer.outputValue=val
    return val

  #This will specifically use sgd to train the model. The steps of gradient descent in general is to perform
  #the forward pass, then backward pass #1 to obtain the gradient of objective with respect to each linear output, then
  #the backward pass #2 to obtain the gradient of objective with respect to each parameter
  def train_sgd(self,train_x,train_y,num_epochs,learning_rate,batch_size=1):
    for epoch in range(num_epochs):
      perm = np.random.permutation(train_x.shape[0])
      # Apply permutation to both x and y
      train_x_shuffled = train_x[perm]
      train_y_shuffled = train_y[perm]

      for i in range(len(train_x_shuffled)):
        x=train_x_shuffled[i].reshape(-1,1)
        y=train_y_shuffled[i].reshape(-1,1)
        #The following is the forward pass
        yHat=self.predict(x,storeValues=True)
        #While we perform the forward pass in each layer we store the output of the layer so we have access to all the intermediate values
        #Now we will perform the backward pass #1 to get the gradient of objective with respect to each linear output

        #backward pass #1

        for i in range(len(self.layers)-1,-1,-1):
          layer=self.layers[i]
          #if we are dealing with the last layer=output layer. For here dj/dz=da/dz*dJ/da
          if (i==len(self.layers)-1):
            activationType=None
            if isinstance(layer.activation, Sigmoid):
              activationType="sigmoid"
            elif isinstance(layer.activation, reLU):
              activationType="relu"
            elif isinstance(layer.activation, Softmax):
              activationType="softmax"
            else:
              pass


            if (layer.loss=="se"):
              if (activationType==None):
                layer.outputGrad=2*(yHat-y)
              else:
                djda=2*(yHat-y)
                dadz=derivOfActWrtLinearInput(activationType,yHat)
                layer.outputGrad=dadz@djda

            elif (layer.loss=="ce"):

              if (activationType=="softmax"):
                #Very special case
                self.layers[i].outputGrad=yHat-y

              else:
                deriv=np.zeros_like(yHat)
                correctClassIndex=np.argmax(y)
                probCorrectClass=yHat[correctClassIndex,0]
                deriv[correctClassIndex,0]=-1*(1/(probCorrectClass))
                djda=deriv
                dadz=derivOfActWrtLinearInput(activationType,yHat)
                layer.outputGrad=dadz@djda

            continue

          else:
            term1=None
            if (layer.activation==None):
              term1=np.eye(layer.outputValue.shape[0])

            elif isinstance(layer.activation, Sigmoid):
              term1=derivOfActWrtLinearInput("sigmoid",layer.outputValue.flatten())
            elif isinstance(layer.activation,reLU):
              term1=derivOfActWrtLinearInput("relu",layer.outputValue.flatten())
            elif isinstance(layer.activation,Softmax):
              term1=derivOfActWrtLinearInput("softmax",layer.activation.outputValue.flatten())
            else:
              pass

            #obtaining the transpose of weight matrix of next linear layer
            term2=np.transpose(self.layers[i+1].weights)

            #this is the gradient of objective with respect to next linear layers output which we calucated in the previous iteration of this backward pass #1
            term3=self.layers[i+1].outputGrad
            self.layers[i].outputGrad=term1@term2@term3

      #backward pass #2
      #now we calculate the gradient of objective with respect to weights and biases in linear layers
        for i in range(len(self.layers)-1,-1,-1):
          layer=self.layers[i]
          derivObjWrtBias=layer.outputGrad
          layer.bias=layer.bias-learning_rate*derivObjWrtBias

          # Special case for the first linear layer
          derivObjWrtWeights=None
          if i==0:
              derivObjWrtWeights=layer.outputGrad@np.transpose(x)
          else:
              derivObjWrtWeights=layer.outputGrad@np.transpose(self.layers[i-1].activation.outputValue)

          layer.weights=layer.weights-learning_rate*derivObjWrtWeights

In [278]:
def squared_error_loss(yHat,y):
      #in Mathematics the squared error loss function is the sum of the squares of the difference between the two vectors

      difference=yHat-y
      return (np.transpose(difference))@difference

In [279]:
def absolute_error_loss(yHat,y):
  difference=yHat-y
  error_vector=np.abs(difference)
  return np.sum(error_vector)

In [280]:
def cross_entropy_loss(yHat,y):
  #Only one component of y will be 1 and the rest will be 0. Therefore to find the index i  where yi =1 we can take the argmax
  #this will be a column vector of the class of each datapoint
  oneIndex=np.argmax(y,axis=1)

  #this will be a column vector of the prob we predicted for the correct class of each datapoint
  probCorrectClass = yHat[np.arange(yHat.shape[0]), oneIndex]

  #vector of losses
  individualLosses=-1*np.log(probCorrectClass)

  return np.sum(individualLosses)


In [281]:
def numerize_output(Y):
  #This function is going to take an output column of strings (classes) and convert it into an output column of numbers from 0 to k-1
  Y_flat=Y.flatten()
  num_points=Y_flat.shape[0]
  #this is a 1d numpy array
  unique_elems=np.unique(Y_flat)
  unique_elems_list=list(unique_elems)
  num_unique_elems=len(unique_elems_list)
  k=num_unique_elems
  unique_elems_list_enum=enumerate(unique_elems_list)


  class_string_to_int_dict={string_class: int_class for (int_class,string_class) in unique_elems_list_enum}

  def string_to_int_class(string):
    #gets you the integer class
    return class_string_to_int_dict[string]

  convert_strings_to_ints=np.vectorize(string_to_int_class)

  y_numbers=convert_strings_to_ints(Y_flat)


  #To get it back into a column vector
  #y_numbers=y_numbers.reshape(-1,1) # Remove this line
  return y_numbers,k

In [282]:
def transform_classification_y_dataset(Y_train,Y_test,k):
  # Y_train and Y_test are going to have the dimensions of some nx1, we want to turn this into nxk where k is the number of classes, the classes go from 0 to k-1
  Y_train_flat=Y_train.flatten()
  Y_test_flat=Y_test.flatten()

  num_train_points=Y_train_flat.shape[0]
  num_test_points=Y_test_flat.shape[0]

  transformed_y_train=np.zeros((num_train_points,k))
  transformed_y_test=np.zeros((num_test_points,k))

  transformed_y_train[np.arange(num_train_points),Y_train_flat]=1
  transformed_y_test[np.arange(num_test_points),Y_test_flat]=1

  return transformed_y_train,transformed_y_test






In [283]:
def extractDatasetComponentsClassification(data):
  np.random.shuffle(data)
  splitIndex=int(0.8*len(data))

  #I must numerise the output class column over here

  Y=data[:,-1]
  #y is nx1
  data[:,-1],k=numerize_output(Y)

  X_train_data=data[:splitIndex,:-1]
  Y_train_data=data[:splitIndex,-1].reshape(-1,1).astype(int)

  X_test_data=data[splitIndex:,:-1]
  Y_test_data=data[splitIndex:,-1].reshape(-1,1).astype(int)

  Y_train_data,Y_test_data=transform_classification_y_dataset(Y_train_data,Y_test_data,k)

  return X_train_data,Y_train_data,X_test_data,Y_test_data



In [284]:
def extractDatasetComponentsRegression(data):
  np.random.shuffle(data)
  splitIndex=int(0.8*len(data))
  X_train_data=data[:splitIndex,:-1]
  Y_train_data=data[:splitIndex,-1].reshape(-1,1)
  X_test_data=data[splitIndex:,:-1]
  Y_test_data=data[splitIndex:,-1].reshape(-1,1)
  return X_train_data,Y_train_data,X_test_data,Y_test_data



In [285]:
#loading and preparing the regression dataset
concrete_df=pd.read_csv("sample_data/concrete.csv")
concrete_data=concrete_df.to_numpy()
ConcreteStrengthX,ConcreteStrengthY,ConcreteStrengthXTest,ConcreteStrengthYTest=extractDatasetComponentsRegression(concrete_data)

#Loading and preparing the classification dataset
iris_df=pd.read_csv("sample_data/iris.csv")
iris_data=iris_df.to_numpy()
iris_data=iris_data[1:,:]
IrisClassX,IrisClassY,IrisClassXTest,IrisClassYTest=extractDatasetComponentsClassification(iris_data)
IrisClassX=IrisClassX.astype(float)
IrisClassY=IrisClassY.astype(float)
IrisClassXTest=IrisClassXTest.astype(float)
IrisClassYTest=IrisClassYTest.astype(float)



In [286]:
def standardize_data(data):
    mean = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data-mean) /std, mean, std

ConcreteStrengthX, concrete_train_mean, concrete_train_std = standardize_data(ConcreteStrengthX)
ConcreteStrengthXTest=(ConcreteStrengthXTest - concrete_train_mean) / concrete_train_std

IrisClassX, iris_train_mean, iris_train_std=standardize_data(IrisClassX)
IrisClassXTest=(IrisClassXTest-iris_train_mean)/iris_train_std



In [287]:
"""
These will be tests for SGD on linear, act, linear, act, linear, act, linear (output) network architecture that was spelled out by the textbook
"""


ListOfLayersConcreteRegression=[LinearLayer(8,3,activation="relu"),LinearLayer(3,3,activation="relu"),LinearLayer(3,1,loss="se")]
ConcreteNetwork=NeuralNetwork(ListOfLayersConcreteRegression)


ListOfLayersIrisClassification=[LinearLayer(4,3,activation="relu"),LinearLayer(3,3,activation="relu"),LinearLayer(3,3,activation="softmax",loss="ce")]
IrisNetwork=NeuralNetwork(ListOfLayersIrisClassification)











In [288]:
ConcreteNetwork.train_sgd(ConcreteStrengthX,ConcreteStrengthY,100,0.02)

IrisNetwork.train_sgd(IrisClassX,IrisClassY,100,0.02)



In [295]:
#I am giving a 8x1 vector as expected to pass through the neural network. I should literally get a continous value spit out
print(ConcreteNetwork.predict(np.array([540,0,0,162,2.5,1040,676,28])))

probabilities=IrisNetwork.predict(np.array([6.1,3.0,4.6,1.4]))
#Note that for this output the first component is the probability of belonging to Iris-setosa class, then Iris-versicolor class,
#then Iris-virginica class


[[37.01465212]]


In [290]:
classes=["iris-setosa","iris-versicolor","iris-virginca"]
for i in range(len(classes)):
  print("Probability of class "+str(i)+" "+classes[i]+ " is "+str(probabilities[i][0]))



Probability of class 0 iris-setosa is 1.31488538050364e-37
Probability of class 1 iris-versicolor is 1.976022586822745e-26
Probability of class 2 iris-virginca is 1.0


In [291]:
yHatConcrete=np.transpose(np.apply_along_axis(ConcreteNetwork.predict,0,(np.transpose(ConcreteStrengthXTest))))
yHatConcrete=yHatConcrete.flatten()
yHatConcrete=yHatConcrete.reshape(yHatConcrete.shape[0],1)

#Number of datapoints in the validation dataset
num_points=yHatConcrete.shape[0]
print("The number of validation data points is: "+str(num_points))

se_loss_array=squared_error_loss(yHatConcrete,ConcreteStrengthYTest)

se_loss=float(se_loss_array)

mse_loss=se_loss/num_points


ae_loss=absolute_error_loss(yHatConcrete,ConcreteStrengthYTest)
mae_loss=ae_loss/num_points


print("Squared error on the validation dataset: " + str(se_loss))
print("Mean squared error on the validation dataset: "+ str(mse_loss))
print("Absolute error on the validation dataset: " + str(ae_loss))
print("Mean absolute error on the validation dataset: "+ str(mae_loss))




The number of validation data points is: 206
Squared error on the validation dataset: 62116.17094120573
Mean squared error on the validation dataset: 301.53481039420257
Absolute error on the validation dataset: 2912.4516508846464
Mean absolute error on the validation dataset: 14.138114810119642


/tmp/ipython-input-3964664427.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  se_loss=float(se_loss_array)


In [292]:

yHatIris=np.transpose(np.apply_along_axis(IrisNetwork.predict,0,(np.transpose(IrisClassXTest))))
yHatIris=yHatIris.reshape(30,3)
#This is nxd
num_points=yHatIris.shape[0]
print("The number of validation data points is: "+str(num_points))




ce_loss_array=cross_entropy_loss(yHatIris,IrisClassYTest)



ce_loss=float(ce_loss_array)

print("Cross entropy loss on the validation dataset: " + str(ce_loss))

average_ce_loss=ce_loss/num_points
print("The average cross entropy loss over the validation dataset: "+str(average_ce_loss))




The number of validation data points is: 30
Cross entropy loss on the validation dataset: 8.432537572327579
The average cross entropy loss over the validation dataset: 0.28108458574425266
